# MPDP: Visco-elasto-plasticity with isotropic hardening

Associated, rate-dependent plasticity. .

Application of GSM MPDP - Dissipation potential for visco-elasticity included


In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSMMPDP
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
sp.init_printing()

### Helmholtz free energy

In [ ]:
E = sp.Symbol(r'E', real=True, nonnegative=True)
K = sp.Symbol(r'K', real=True)
f_c = sp.Symbol(r'f_\mathrm{c}')
eta_vp = sp.Symbol(r'\eta_\mathrm{vp}', real=True, nonnegative=True)
eta_ve = sp.Symbol(r'\eta_\mathrm{ve}', real=True, nonnegative=True)


mparams = (E, K, f_c, eta_vp, eta_ve)

# ## External state variables

eps = sp.Symbol(r'\varepsilon', real=True)
eps_a = sp.Matrix([eps])
sig = sp.Symbol(r'\sigma', real=True)
sig_a = sp.Matrix([sig])

# ## Internal state variables

eps_p = sp.Symbol(r'\varepsilon^\mathrm{vp}', real=True)
eps_p_a = sp.Matrix([eps_p])
sig_p = sp.Symbol(r'\sigma^\mathrm{vp}', real=True)
sig_p_a = sp.Matrix([sig_p])

eps_v = sp.Symbol(r'\varepsilon^\mathrm{ve}', real=True)
eps_v_a = sp.Matrix([eps_v])
sig_v = sp.Symbol(r'\sigma^\mathrm{ve}', real=True)
sig_v_a = sp.Matrix([sig_v])

z = sp.Symbol(r'z', real=True, nonnegative=True)
z_a = sp.Matrix([z])
Z = sp.Symbol(r'Z', real=True, nonnegative=True)
Z_a = sp.Matrix([Z])

# ## Free energy potential
eps_el = eps - eps_v - eps_p
U_e_ = sp.Rational(1,2) * E * eps_el**2
U_p_ =  sp.Rational(1,2) * K * z**2
F_ = U_e_ + U_p_

# ## Flow potential
f_ = sp.sqrt(sig_p**2) - (f_c + Z)
phi_ext_ = 0

Eps_vars = (eps_v_a, eps_p_a, z_a)
Sig_vars = (sig_v_a, sig_p_a, Z_a)
Sig_signs =  (-1, -1, 1)


### GSM drivers

In [ ]:
gsm_vevp = GSMMPDP(
    name = 'gsm_F_1d_mpdp_vevp_lih',
    diff_along_rates = False,
    u_vars = eps_a,
    sig_vars = sig_a,
    m_params = mparams,
    Eps_vars = Eps_vars,
    Sig_vars = Sig_vars,
    Sig_signs = Sig_signs,
    F_expr = F_,
    f_expr = f_,
)
dot_eps_p = gsm_vevp.dot_Eps[1, 0]
f_d_ = gsm_vevp.f_expr - eta_vp * dot_eps_p
gsm_vevp.f_expr = f_d_

dot_eps_ve = gsm_vevp.dot_Eps[0, 0]
sig_ve = gsm_vevp.Sig[0, 0]
gsm_vevp.h_k = [eta_ve * dot_eps_ve - sig_ve]

In [ ]:
gsm_vevp.F_expr

In [ ]:
gsm_vevp.phi_

In [ ]:
gsm_vevp.Sig_, gsm_vevp.phi_.diff(gsm_vevp.Sig.as_explicit()), gsm_vevp.Y_, gsm_vevp.Phi_

In [ ]:
(gamma_mech, L_, dL_dS_, dL_dS_A_, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, Ox, Ix), Sig_n1, f_n1, R_n1, dR_dA_n1_OI = gsm_vevp.Sig_f_R_dR_n1


In [ ]:
gamma_mech

In [ ]:
L_

In [ ]:
Sig_n1

In [ ]:
dL_dS_

In [ ]:
R_n1_subs = R_n1.replace(sp.sign, lambda x: 1)
R_n1_subs

In [ ]:
delta_A.T

In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
mparams_epd = {E:1, K:10, f_c:1, eta_vp:1, eta_ve:1}
#mparams_epd = {E:1, K:10, f_c:1, eta_vp:0.01, eta_ve:1}
margs = [mparams_epd[p] for p in mparams]
n_t = 10
n_I = 1
eps_max = 6
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_vevp.n_Eps), dtype=np.float64)
        # return (t_t[:n_t], eps_ta, sig_ta, Eps_t, Sig_t, iter_t, 
        #         lam_t, (d_t_t[:n_t], d_eps_ta[:n_t]))
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_vevp.get_response(eps_n_t, t_t, 100, *margs)
eps_ve_t, eps_vp_t, z_t = Eps_t[:, 0, 0], Eps_t[:, 0, 1], Eps_t[:, 0, 2]
eps_t = eps_ta[:, 0]
eps_t.shape, eps_ve_t.shape, eps_vp_t.shape

In [ ]:
fig, (ax_sig, ax_eps_v) = plt.subplots(1, 2, figsize=(12, 4))

# Plot viscoplastic strain and damage along time
ax_sig.plot(eps_t, Sig_t[:,:,0], color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_ve_t - eps_vp_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon_\mathrm{e}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_eps_v.plot(t_t, eps_ve_t, label=r'$\varepsilon_\mathrm{ve}$', color='royalblue')
ax_eps_v.plot(t_t, eps_ve_t + eps_vp_t, label='$\varepsilon_\mathrm{vp}$', color='orange')
ax_eps_v.plot(t_t, eps_t, label='$\varepsilon$', color='black')

ax_eps_v.set_xlabel(r'$t$')
ax_eps_v.set_ylabel(r'$\varepsilon$')

plt.tight_layout()
plt.show()

In [ ]:
mparams_epd = {E:10000, K:10000, f_c:1, eta_vp:100, eta_ve:10000}
margs = [mparams_epd[p] for p in mparams]

d_t_val = 0.01 # Time step in seconds
final_time = 20  # Final time in secondsfinal_time = 1

# Generate cyclic sinusoidal loading history
t_t = np.arange(0, final_time, d_t_val)
eps_t_2_cycles = 0.01 * np.sin(2 * np.pi * 2 * t_t / final_time)  # 2 cycles
eps_t_4_cycles = 0.01 * np.sin(2 * np.pi * 4 * t_t / final_time)  # 4 cycles
Eps_n_Ib = np.zeros((n_I, gsm_vevp.n_Eps), dtype=np.float64)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_vevp.get_response(eps_t_4_cycles, t_t, 100, *margs)
eps_ve_t, eps_vp_t, z_t = Eps_t[:, 0, 0], Eps_t[:, 0, 1], Eps_t[:, 0, 2]
eps_t = eps_ta[:, 0]
eps_t.shape, eps_ve_t.shape, eps_vp_t.shape

In [ ]:
fig, (ax_sig, ax_eps_v) = plt.subplots(1, 2, figsize=(12, 4))

# Plot viscoplastic strain and damage along time
ax_sig.plot(eps_t, Sig_t[:,:,0], color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_ve_t - eps_vp_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon_\mathrm{e}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_eps_v.plot(t_t, eps_ve_t, label=r'$\varepsilon_\mathrm{ve}$', color='royalblue')
ax_eps_v.plot(t_t, eps_ve_t + eps_vp_t, label='$\varepsilon_\mathrm{vp}$', color='orange')
ax_eps_v.plot(t_t, eps_t, label='$\varepsilon$', color='black')

ax_eps_v.set_xlabel(r'$t$')
ax_eps_v.set_ylabel(r'$\varepsilon$')

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib widget
mparams = {E:1, K:0.1, f_c:1, eta_vp:1, eta_ve:0.5}
margs = [mparams[p] for p in mparams]
n_t = 200
n_I = 1
eps_max = 6
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
relax_t = np.linspace(3, 10, 7*n_t)[1:]
t_t = np.hstack([t_t, np.linspace(1, 2, n_t)[1:], np.linspace(2, 3, n_t)[1:], relax_t])
eps_n_t = np.hstack([eps_n_t, eps_n_t[-1] - 0 * eps_n_t[1:], eps_n_t[-1::-1], np.zeros_like(relax_t)])
fig, ax = plt.subplots(1,1)
ax.plot(eps_n_t)

In [ ]:
# mparams_epd = {E:1, K:0.1, f_c:1, eta_vp:1, eta_ve:1}
# margs = [mparams_epd[p] for p in mparams]
# n_t = 200
# n_I = 1
# eps_max = 6
# t_t = np.linspace(0, 1, n_t)
# t_t = np.hstack([t_t, t_t[-1] + t_t[1:]])
# eps_n_t = np.linspace(0, eps_max, n_t)
# eps_n_t = np.hstack([eps_n_t, eps_n_t[-1] * np.ones_like(eps_n_t[:-1])])
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_vevp.get_response(eps_n_t, t_t, 100, *margs)
eps_ve_t, eps_vp_t, z_t = Eps_t[:, 0, 0], Eps_t[:, 0, 1], Eps_t[:, 0, 2]
eps_t = eps_ta[:, 0]
eps_t.shape, eps_ve_t.shape, eps_vp_t.shape

In [ ]:
fig, (ax_sig, ax_eps_v) = plt.subplots(1, 2, figsize=(12, 4))

# Plot viscoplastic strain and damage along time
ax_sig.plot(eps_t, Sig_t[:,:,0], color='black', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_ve_t - eps_vp_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon_\mathrm{e}$')
ax_sig.plot(eps_t-eps_vp_t, Sig_t[:,:,0], ls='dashed', color='royalblue', label=r'$\varepsilon_\mathrm{e} + \varepsilon_\mathrm{ve}$')
ax_sig.plot(eps_t-eps_ve_t, Sig_t[:,:,0], ls='dashed', color='orange', label=r'$\varepsilon_\mathrm{e} + \varepsilon_\mathrm{vp}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_eps_v.plot(t_t, eps_ve_t, label=r'$\varepsilon_\mathrm{ve}$', color='royalblue')
ax_eps_v.plot(t_t, eps_vp_t, label=r'$\varepsilon_\mathrm{vp}$', color='orange')
ax_eps_v.plot(t_t, eps_t, label=r'$\varepsilon$', color='black')

ax_eps_v.set_xlabel(r'$t$')
ax_eps_v.set_ylabel(r'$\varepsilon$')
ax_eps_v.legend()

plt.tight_layout()
plt.show()

In [ ]:
mparams_epd = {E:1, K:0, f_c:1, eta_vp:0, eta_ve:0.1}
margs = [mparams_epd[p] for p in mparams]
eps_n_I = np.array([0, 1.0, 1.0, -1.0, 0, 0], dtype=np.float64)
d_eps_I = np.array([0.1, -0.1, 0.1, -0.1, 2, -2], dtype=np.float64)
n_I = eps_n_I.shape[0]
d_t = 1
Eps_n_Ib = np.zeros((n_I, gsm_vevp.n_Eps), dtype=np.float64)

Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I = gsm_vevp.get_state_n1(eps_n_I, d_eps_I, d_t, Eps_n_Ib, 30, *margs )
Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I